# ECON 490: Within Group Analysis (7)

## Prerequisites 

1. Be able to effectively use Stata do-files and generate log-files.
2. Be able to change your directory so that Stata can find your files.
3. Import datasets in csv and dta format. 
4. Save data files. 

## Learning Outcomes

1. Create new variables using the command `egen`.
2. Know when to use the pre-command `by` and when to use `bysort`. 
3. Use the command `collapse` to create a new data set of summary statistics.
3. Change a panel data set to a cross-sectional data set using the command `reshape`. 

## 7.1 Introduction to Working Within Groups

There are times when you need to consider workers as a group. Consider some of the following examples:

- You would like to know the average wages of workers by educational grouping, in each year of the data. 
- You would like to know the standard deviation of men and women's earnings, in each geographic region in the data. 
- You would like to know the top quintile of wealth, by birth cohort. 

This module will show you how to calculate these statistics using the fake data data set introduced in the previous lecture. Recall that this data set is simulating information of workers in the years 1982-2012 in a fake country where a training program was introduced in 2003 to boost their earnings.  

Let's begin by loading that data set into Stata:

In [ ]:
clear *

use "fake_data.dta", clear

## 7.2 Generating Variables using `generate`

When we are working on a particular project, it is important to know how to create variables that are computed for a group rather than an individual or an observation. For instance, we may have a data set that is divided by individual and by year. We might want the variables to show us the statistics of a particular individual throughout the years or the statistics of all individuals each year.

Stata provides functionality to easily compute such statistics. The key to this analysis is the pre-command `by`, and the only requisite to using this is to ensure data is sorted the correct way.

Let's take a look at our data by using the `browse` command we learned in [Module #]().

In [ ]:
%browse 10

We can tell here that the data is sorted by the variable *workerid*. 


We use the pre-command `by` alongside the command `generate` to develop these group compounded variables. If we use variables other than *workerid* (the variable by which the data is sorted) to group our new variable, we will not be able to generate the new variable.

When we run the command below Stata will produce this error. 

In [ ]:
cap drop var_one 
by year: gen var_one = 1 

If we want to group by year, Stata expects us to sort the data such that all observations corresponding to the same year are next to each other. We can use the `sort` command as follows.

In [ ]:
sort year 

In [ ]:
%browse 10

Let's try the command above again, now with the sorted data.

In [ ]:
cap drop var_one 
by year: gen var_one = 1 

Now that the data is sorted by year, the code works! 

We could have also used the pre-command `bysort` instead of `by`. When we do this we can skip the command to sort the data. Everything is done in one step!

Let's sort the data, so it is reverted back to the same ordering scheme as when we started, and generate our new variable again. 

In [ ]:
sort workerid year 

In [ ]:
cap drop var_one 
bysort year: gen var_one = 1 

The variable we have created is not interesting by any means. It simply takes the value of 1 everywhere. In fact, we haven't done anything that we couldn't have done with `gen var_one=1`. We can see this by using the `summary` command.

In [ ]:
su var_one

You may not be aware, but Stata records the observation number as a hidden variable (a scalar) called *_n* and the total number of observations as *_N*.

Let's take a look at these by creating new two variables: one that is the observation number and one that is the total number of observations.

In [ ]:
cap drop obs_number 
gen obs_number = _n 

cap drop tot_obs
gen tot_obs = _N

In [ ]:
%browse 10

As expected, the numbering of observations is sensitive to the way that the data is sorted! The cool thing is that whenever we use the pre-command `by`, the scalars `_n` and `_N` record the observation number and total number of observations for each group separately.

In [ ]:
cap drop obs_number 
bysort workerid: gen obs_number = _n 

cap drop tot_obs
bysort workerid: gen tot_obs = _N

In [ ]:
%browse 10

As we can see, some workers are observed only 2 times in the data (they were only surveyed in two years), whereas other workers are observed 8 times (they were surveyed in 8 years). By knowing (and recording in a variable) the number of times a worker has been observed, we can do some analysis based on this information. For example, in some cases you might be interested in keeping only workers who are observed across all time periods. In this case, you could use the command:

In [ ]:
keep if tot_obs==8

In [ ]:
%browse 10

## 7.3 Generating Variables Using Extended Generate

The command `egenerate` is used whenever we want to create variables which require access to some functions (e.g. mean, standard deviation, min).  The basic syntax works as follows: 

```stata
 bysort groupvar: egen new_var = function() , options
```

Let's see an example where we create a new variable called *avg_earnings* which is the mean of earnings for every worker. We will need to reload our data since we dropped many observations above when we used the `keep` command.

In [ ]:
clear *
use "fake_data.dta", clear

In [ ]:
cap drop avg_earnings
bysort workerid: egen avg_earnings = mean(earnings)

In [ ]:
cap drop total_earnings
bysort workerid: egen total_earnings = total(earnings)

By definition, these commands will create variables that use information across different observations. You can check the list of available functions by writing `help egen` in the Stata command window. 

In this documentation, you will notice that there are some functions that do not allow for `by`. For example, suppose we want to create the total sum across different variables in the same row. 

In [ ]:
cap drop sum_of_vars
egen sum_of_vars = rowtotal(start_year region treated)

The variable we are creating for the example has no particular meaning, but what we need to notice is that the function `rowtotal()` only sums the non-missing values in our variables. This means that if there is a missing value in any of the three variables, the sum only occurs between the two variables that do not have the missing value. We could also write this command as `gen sum_of_vars = start_year + region + treated`; however, if there is a missing value (`.`) in *start_year*,  *region* or *treated*, then the generated value for *sum_of_vars* will also be a missing value. The answer lies in the missing observations. If we sum any number with a missing value (`.`), then the sum will also be missing.

We can also use `by` with a list of variables. Here will use *year* and *region* in one command.

In [ ]:
cap drop regionyear_earnings
bysort year region : egen regionyear_earnings = total(earnings)

What this command gives us is a new variable that records total earnings in each region for every year.

## 7.4 Collapsing Data

We can also compute statistics at some group level with the `collapse` command. Collapse is extremely useful whenever we want to apply sample weights to our data (we will learn more about this in [Module 12](econometrics/econ490-stata/12_Linear_Reg.ipynb). Sample weights cannot be applied using `egen` but are often extremely important when using micro data. Those weights allow us to manipulate our data to better reflect the composition of the data when the authority that collected the data might have over sampled some segments of the population. 

The syntax is 

```stata
collapse (statistic1) new_name = existing_variable (statistic2) new_name2 = existing_variable2 ... [pweight =     weight_variable], by(group) 
```

You can obtain a list of possible statistics by running the command `help collapse`. You can also learn more about using weights by typing `help weight`.

Let's suppose we want to create a data set at the region-year level using information in the current data set, but we want to use the sample weights that were provided with our data. First, we decide which statistics we want to keep from the original data set. For the sake of explanation, let's suppose we want to keep average earnings, the variance of earnings, and the total employment. We write the following:

In [ ]:
collapse (mean) avg_earnings = earnings (sd) sd_earnings = earnings (count) tot_emp = earnings, by(region year)

In [ ]:
%browse 10

<div class="alert alert-block alert-warning">
    
<b>Warning:</b> When you use `collapse`, Stata produces a new data set with the results and in the process drops the data set that was loaded at the time the command was run. If you need to keep that data, be certain to save the file before you run this command. 
</div>

## 7.5 Reshaping 

We have collapsed our data and so we need to import the data again to gain access to the full data set.

In [ ]:
clear *

use "fake_data.dta", clear

Notice that the nature of this particular data set is panel form; individuals have been followed over many years. Sometimes we are interested in working with a cross section (i.e. we have 1 observation per worker which includes all of the years). Is there a simple way to go back and forth between these two? Yes!

The command's name is `reshape` and has two main forms: `wide` and `long`. The former is related to a cross-sectional nature, whereas the latter relates to the usual panel nature. 

Suppose we want to record the earnings of workers while keeping the information across years.

In [ ]:
reshape wide earnings region age start_year sample_weight, i(workerid) j(year)

<div class="alert alert-warning">

**Warning:** This command acts on all of the variables in your data set. If you don't include them in the list, Stata will assume that they do not vary across *i* (in this case workers). If you don't check this beforehand, you may get an error message.

</div>

In [ ]:
%browse 10

There are so many missing values in the data! Should we worry? Not at all. As a matter of fact, we learned at the beginning of this module that many workers are not observed across all years. That's what these missing values are representing. 

Notice that the variable _year_ which was part of the command line (the `j(year)` part) has disappeared. We now have one observation per worker, with their information recorded across years in a cross-sectional way. 

How do we go from a `wide` data set to a regular panel form? We need to indicate the prefix in the variables, which are formally known as `stubs` in the Stata lingo, and use the `reshape long` command. When we write `j(year)` it will create a new variable called _year_.

In [ ]:
reshape long earnings region age  start_year sample_weight, i(workerid) j(year) 

In [ ]:
%browse 10

Notice that we now have an observation for every worker in every year, although we know some workers are only observed in a subset of these. This is known as a **balanced panel**.  

To retrieve the original data set, we get rid of such observations with missing values.

In [ ]:
keep if !missing(earnings)

In [ ]:
%browse 10

## 7.6 Wrap Up 

In this module, you have developed some very useful skills that will help you explore data sets. Namely, these skills will help you both prepare your data for empirical analysis (i.e. turning cross sectional data into panel data) and create summary statistics that you can use to illustrate your results. In the next module, we will look at how to work with multiple data sets simultaneously and merge them into one.

## References

[Reshape data from wide format to long format](https://www.youtube.com/watch?v=Bx9kVdkr9oY) <br>
[(Non StataCorp) How to group data in STATA with SORT and BY](https://www.youtube.com/watch?v=nEOyH0AFKHc)